# ETL 1.0 (PostgreSQL)

In [1]:
import pandas as pd
import sqlalchemy
import psycopg2
from sqlalchemy import create_engine, text
import numpy as np

/Users/isaiaherb/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/isaiaherb/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
db_name = "police"
db_user = "isaiaherb"
db_password = "sharkboy1"
db_host = "localhost"
db_port = "5432"  
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

## Extract, Transform, and Load Postgres Tables

### 'State' table

In [12]:
state_data = {
    'state_id': range(1, 51),  
    'state_name': [
        'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
        'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
        'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
        'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
        'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
        'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
        'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
        'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
        'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
        'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
    ],
    'state_abbreviation': [
        'AL', 'AK', 'AZ', 'AR', 'CA',
        'CO', 'CT', 'DE', 'FL', 'GA',
        'HI', 'ID', 'IL', 'IN', 'IA',
        'KS', 'KY', 'LA', 'ME', 'MD',
        'MA', 'MI', 'MN', 'MS', 'MO',
        'MT', 'NE', 'NV', 'NH', 'NJ',
        'NM', 'NY', 'NC', 'ND', 'OH',
        'OK', 'OR', 'PA', 'RI', 'SC',
        'SD', 'TN', 'TX', 'UT', 'VT',
        'VA', 'WA', 'WV', 'WI', 'WY'
    ]
}

states_df = pd.DataFrame(state_data)
states_df['state_id'] = states_df['state_id'].astype(int)

new_row={
    'state_id': 51,
    'state_name': 'United States',
    'state_abbreviation': 'US'
}
new_row_df=pd.DataFrame([new_row])
states_df = pd.concat([states_df,new_row_df], ignore_index=True)

with engine.connect() as conn:
    try:
        conn.execute(
            text("""
                CREATE TABLE IF NOT EXISTS state (
                    state_id SERIAL PRIMARY KEY,
                    state_name VARCHAR(50) UNIQUE,
                    state_abbreviation VARCHAR(10) UNIQUE
                )
            """)
        )
        print("'State' table created successfully.")
        # for index, row in states_df.iterrows():
        #     conn.execute(
        #         text("""
        #             INSERT INTO state (
        #                 state_id,
        #                 state_name,
        #                 state_abbreviation
        #             ) VALUES (
        #                 :state_id,
        #                 :state_name,
        #                 :state_abbreviation
        #             )
        #         """),
        #         {
        #             "state_id": row['state_id'],
        #             "state_name": row['state_name'],
        #             "state_abbreviation": row['state_abbreviation']
        #         }
        #     )
        # print("Data loaded into 'state' table successfully.")
    except Exception as e:
        print(f"Error: {e}")

'State' table created successfully.


### 'City' table

In [65]:
df1 = pd.read_csv('data/shootings_wash_post.csv')
df1 = df1[['city', 'state']]
df1 = df1.rename(columns={'city': 'city_name', 'state': 'state_abbreviation'})

df2 = pd.read_csv('data/police_info/budgets.csv')
df2 = pd.DataFrame(df2['city_name'])
df2[['city_name', 'state_abbreviation']] = df2['city_name'].str.split(': ', expand=True)

df3 = pd.read_csv('data/police_killings_MPV.csv', dtype={'column21': 'str', 'column22': 'str', 'column23': 'str'})
df3 = df3[['City', 'State']]
df3 = df3.rename(columns={'City': 'city_name', 'State': 'state_abbreviation'})

df4 = pd.read_csv('data/deaths_arrests.csv')
df4 = df4[['State', 'City']]
df4 = df4[df4['State'] != 'United States']
df4 = df4.rename(columns={'State': 'state_name', 'City': 'city_name'})

df5 = pd.read_csv('data/demographics/politics_538.csv')
df5 = df5[['Area']]
df5[['city_name', 'state_name']] = df5['Area'].str.split(', ', expand=True, n=1)
df5 = df5.drop(columns=['Area'])

df6 = pd.read_csv('data/demographics/poverty_census_bureau.csv')
df6 = df6[['Geographic Area Name']]
df6[['city_name', 'state_name']] = df6['Geographic Area Name'].str.split(', ', expand=True, n=1)
df6['state_abbreviation'] = df6['Geographic Area Name'].str.extract(r',\s*([A-Z]{2})')
df6 = df6.drop(columns=['state_name','Geographic Area Name'])

df7 = pd.read_csv('data/demographics/education_census_bureau.csv')
df7 = df7[['Geographic Area Name']]
df7[['city_name', 'state_name']] = df7['Geographic Area Name'].str.split(', ', expand=True, n=1)
df7['state_abbreviation'] = df7['Geographic Area Name'].str.extract(r',\s*([A-Z]{2})')
df7 = df7.drop(columns=['state_name','Geographic Area Name'])

cities = pd.concat([df1, df2, df3, df4, df5, df6, df7], axis=0)
cities = cities.drop_duplicates(subset='city_name')
cities.reset_index(drop=True, inplace=True)
cities.dropna()
cities=cities.drop(columns=['state_name'])

cities_df = pd.merge(cities, states_df, on='state_abbreviation', how='left')
cities_df = cities_df.dropna()
cities_df['state_id'] = cities_df['state_id'].astype(int)

cities_df.index = cities_df.index + 1
cities_df['city_id'] = cities_df.index

# with engine.connect() as conn:
#     try:
#         conn.execute(
#             text("""
#                 CREATE TABLE IF NOT EXISTS city (
#                     city_id SERIAL PRIMARY KEY,
#                     city_name VARCHAR(50) UNIQUE,
#                     state_id INT,
#                     CONSTRAINT fk_city_state FOREIGN KEY (state_id) REFERENCES state (state_id)
#                 )
#             """)
#         )
#         print("'City' table created successfully.")
#         for index, row in cities_df.iterrows():
#             conn.execute(
#                 text("""
#                     INSERT INTO city (
#                         city_name,
#                         state_id
#                     ) VALUES (
#                         :city_name,
#                         :state_id
#                     )
#                 """),
#                 {
#                     "city_name": row['city_name'],
#                     "state_id": row['state_id']
#                 }
#             )
#         print("Data loaded into 'city' table successfully.")
#     except Exception as e:
#         print(f"Error: {e}")

/var/folders/1q/xp3s_g4562n85d_j0xm_sp080000gn/T/ipykernel_19842/2990021244.py:9: DtypeWarning: Columns (21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('data/police_killings_MPV.csv', dtype={'column21': 'str', 'column22': 'str', 'column23': 'str'})


,city_name,state_abbreviation,state_id,state_name,city_id
4093,Wooster,OH,35,Ohio,4093
4094,Yankton,SD,41,South Dakota,4094
4095,York-Hanover,PA,38,Pennsylvania,4095
4096,Youngstown-Warren-Boardman,OH,35,Ohio,4096
4097,Zapata,TX,43,Texas,4097


### 'Spending' table

In [38]:
spending_df=pd.read_csv('data/demographics/dqs_table_88_3.csv')

spending_df = spending_df.drop(columns=['Inter-governmental','Total.1'])
spending_df['State']=spending_df['State'].str.replace(r'\.{2,}', '', regex=True)

spending_df = spending_df.rename(columns={
    'State':'state_name',
    'Total': 'total',
    'Basic Education': 'basic_education_percent',
    'Higher Education': 'higher_education_percent',
    'Public Welfare': 'public_welfare_percent',
    'Health and Hospitals': 'public_health_percent',
    'Highways': 'highways_percent',
    'Police': 'police_percent',
    'Other': 'other_percent',
    'Personal Income ($M)': 'total_personal_income_millions'
})

spending_df['total_personal_income_millions'] = spending_df['total_personal_income_millions'].str.replace(',', '').astype(float)
merged_df=pd.merge(spending_df, states_df, on='state_name', how='left')
merged_df = merged_df.dropna()
merged_df['state_id'] = merged_df['state_id'].astype(int)

with engine.connect() as conn:
    try:
        conn.execute(
            text("""   
                CREATE TABLE IF NOT EXISTS spending (
                    spending_id SERIAL PRIMARY KEY,
                    state_id INT NOT NULL,
                    basic_education_percent NUMERIC,
                    higher_education_percent NUMERIC,
                    public_welfare_percent NUMERIC,
                    public_health_percent NUMERIC,
                    highways_percent NUMERIC,
                    police_percent NUMERIC,
                    other_percent NUMERIC,
                    total_personal_income_millions NUMERIC,
                    CONSTRAINT fk_location_spending_state FOREIGN KEY (state_id) REFERENCES state (state_id)
                )  
            """)
        )
        print("Table created succesfully")
        for index, row in merged_df.iterrows():
            conn.execute(
                text("""
                    INSERT INTO spending (
                        state_id,
                        basic_education_percent,
                        higher_education_percent,
                        public_welfare_percent,
                        public_health_percent,
                        highways_percent,
                        police_percent,
                        other_percent,
                        total_personal_income_millions
                    ) VALUES (
                        :state_id,
                        :basic_education_percent,
                        :higher_education_percent,
                        :public_welfare_percent,
                        :public_health_percent,
                        :highways_percent,
                        :police_percent,
                        :other_percent,
                        :total_personal_income_millions
                    )
                """),
                {
                    "state_id": row['state_id'],
                    "basic_education_percent": float(row['basic_education_percent']),
                    "higher_education_percent": float(row['higher_education_percent']),
                    "public_welfare_percent": float(row['public_welfare_percent']),
                    "public_health_percent": float(row['public_health_percent']),
                    "highways_percent": float(row['highways_percent']),
                    "police_percent": float(row['police_percent']),
                    "other_percent": float(row['other_percent']),
                    "total_personal_income_millions": float(row['total_personal_income_millions'])
                }
            )
        print("Data loaded successfully")
    except Exception as e:
        print(f"Error: {e}")

Table created succesfully
Data loaded successfully


### 'Racial Demographics' table

In [72]:
races_df=pd.read_csv('data/deaths_arrests.csv')
races_df=races_df[['State', 'City', 
       'Total', 'Black',
       'White', 'Amer. Indian', 'Asian', 'Hawaiian', 'Asian/Pacific Islander',
       'Other', 'Two or\nmore races', 'Hispanic',
       ]]
races_df=races_df.rename(columns={
    'State': 'state_name', 
    'City': 'city_name',
    'White': 'white',
    'Black': 'black', 
    'Hispanic': 'hispanic', 
    'Hawaiian': 'hawaiian',
    'Asian': 'asian',
    'Amer. Indian': 'american_indian',
    'Asian/Pacific Islander': 'asian_pacific_islander', 
    'Other':'other',
    'Two or\nmore races': 'multiple_races', 
    'Total': 'total_population'
})
races_df=pd.merge(races_df,cities_df,on='city_name',how='left')

columns_to_convert = [
    'total_population', 'white', 'black', 'hispanic', 'hawaiian', 
    'asian', 'american_indian', 'asian_pacific_islander', 
    'other', 'multiple_races'
]

for column in columns_to_convert:
    races_df[column] = races_df[column].str.replace(',', '').astype(float)
    
races_df=races_df.dropna()

with engine.connect() as conn:
    try:
        conn.execute(
            text("""
                CREATE TABLE IF NOT EXISTS racial_demographics (
                  id SERIAL PRIMARY KEY,
                  city_id INT NOT NULL,
                  state_id INT NOT NULL,
                  total_population NUMERIC,
                  white NUMERIC,
                  black NUMERIC,
                  hispanic NUMERIC,
                  hawaiian NUMERIC,
                  asian NUMERIC,
                  american_indian NUMERIC,
                  asian_pacific_islander NUMERIC,
                  other NUMERIC,
                  multiple_races NUMERIC,
                  CONSTRAINT fk_city_id FOREIGN KEY (city_id) REFERENCES city(city_id)
                )
            """)
        )
        print("Table 'racial_demographics' created succesfully.")
        for index, row in races_df.iterrows():
            conn.execute(
                text("""
                    INSERT INTO racial_demographics (
                        city_id,
                        state_id,
                        total_population,
                        white,
                        black,
                        hispanic,
                        hawaiian,
                        asian,
                        american_indian,
                        asian_pacific_islander,
                        other,
                        multiple_races
                    ) VALUES (
                        :city_id,
                        :state_id,
                        :total_population,
                        :white,
                        :black,
                        :hispanic,
                        :hawaiian,
                        :asian,
                        :american_indian,
                        :asian_pacific_islander,
                        :other,
                        :multiple_races
                    )
                """),
                {
                    "city_id": int(row['city_id']),
                    "state_id": int(row['state_id']),
                    "total_population": float(row['total_population']),
                    "white": float(row['white']),
                    "black": float(row['black']),
                    "hispanic": float(row['hispanic']),
                    "hawaiian": float(row['hawaiian']),
                    "asian": float(row['asian']),
                    "american_indian": float(row['american_indian']),
                    "asian_pacific_islander": float(row['asian_pacific_islander']),
                    "other": float(row['other']),
                    "multiple_races": float(row['multiple_races'])
                }
            )           
        print("'Data loaded into 'racial_demographics' table successfully.")
    except Exception as e:
        print(f"Error: {e}")    

Table 'racial_demographics' created succesfully.
'Data loaded into 'racial_demographics' table successfully.


### 'Police Violence' table

In [98]:
deaths_arrests = pd.read_csv('data/deaths_arrests.csv')

deaths_arrests.rename(columns={
    'State': 'state_name',
    'City': 'city_name',
    'PD': 'dept_name',
    'Black People Killed by Police (1/1/2013-12/31/2019)': 'black_killed_by_police',
    'Hispanic People Killed by Police (1/1/2013-12/31/2019)': 'hispanic_killed_by_police',
    'Native American People Killed by Police (1/1/2013-12/31/2019)': 'american_indian_or_alaska_native_killed_by_police',
    'Asian People Killed by Police (1/1/2013-12/31/2019)': 'asian_killed_by_police',
    'Pacific Islanders Killed by Police (1/1/2013-12/31/2019)': 'pacific_islander_killed_by_police',
    'White People Killed by Police (1/1/2013-12/31/2019)': 'white_killed_by_police',
    'Unknown Race People Killed by Police (1/1/2013-12/31/2019)': 'unknown_race_killed_by_police',
    'All People Killed by Police (1/1/2013-12/31/2019)': 'total_killed_by_police',
    'Black-White Dissimilarity Index (2010)': 'black_white_dissimilarity_index_2010',
    'Murder and\nnonnegligent manslaughter': 'murder_and_nonnegligent_manslaughter',
    'Murder Rate': 'murder_rate',
    'Avg Annual Police Homicide Rate': 'avg_annual_police_homicide_rate',
    'Avg Annual Police Homicide Rate for Black People': 'avg_annual_police_homicide_rate_black',
    'Avg Annual Police Homicide Rate for White People': 'avg_annual_police_homicide_rate_white',
    'Avg Annual Police Homicide Rate for Hispanic People': 'avg_annual_police_homicide_rate_hispanic',
    'Black-White Disparity': 'black_white_disparity',
    'Hispanic-White Disparity': 'hispanic_white_disparity',
    'Violent crimes 2013 (if reported by agency)': 'violent_crimes_2013',
    'Violent crimes 2014 (if reported by agency)': 'violent_crimes_2014',
    'Violent crimes 2015 (if reported by agency)': 'violent_crimes_2015',
    'Violent crimes 2016 (if reported by agency)': 'violent_crimes_2016',
    'Violent crimes 2017 (if reported by agency)': 'violent_crimes_2017',
    'Violent crimes 2018 (if reported by agency)': 'violent_crimes_2018',
    'Average Violent Crimes Reported (2013-17)': 'avg_violent_crimes_reported_2013_2017',
    'Violent Crime Rate': 'violent_crime_rate',
    '2013 Total Arrests (UCR Data)': 'total_arrests_2013',
    '2014 Total Arrests': 'total_arrests_2014',
    '2015 Total Arrests': 'total_arrests_2015',
    '2016 Total Arrests': 'total_arrests_2016',
    '2017 Total Arrests': 'total_arrests_2017',
    '2018 Total Arrests': 'total_arrests_2018',
    'Estimated Average Arrests per Year': 'estimated_avg_arrests_per_year',
    'Killings by Police per 10k Arrests': 'killings_by_police_per_10k_arrests'
}, inplace=True)

columns_to_drop = [
    'Total', 'Black', 'White', 'Amer. Indian', 'Asian', 'Hawaiian', 'Asian/Pacific Islander',
    'Other', 'Two or\nmore races', 'Hispanic','total_arrests_2013','total_arrests_2014','total_arrests_2015','total_arrests_2016',
    'total_arrests_2017','total_arrests_2018','violent_crimes_2013','violent_crimes_2014','violent_crimes_2015',
    'violent_crimes_2016','violent_crimes_2017','violent_crimes_2018'
]

deaths_arrests.drop(columns=columns_to_drop, inplace=True)
deaths_arrests = deaths_arrests.dropna(subset=['state_name', 'city_name'])
deaths_arrests=pd.merge(deaths_arrests,cities_df,on='city_name',how='left')
deaths_arrests = deaths_arrests[deaths_arrests['state_name_x'] != 'United States']
deaths_arrests.replace(['#DIV/0!', 'No White People were Killed by Police', 'No People were Killed by Police'], np.nan, inplace=True)
deaths_arrests=deaths_arrests.dropna(subset=['state_id','city_id'])

with engine.connect() as conn:
    try:
        conn.execute(
            text("""   
                CREATE TABLE IF NOT EXISTS police_violence (
                    police_violence_id SERIAL PRIMARY KEY,
                    dept_name VARCHAR(50),
                    city_id INT NOT NULL,
                    state_id INT NOT NULL,
                    asian_killed_by_police NUMERIC,
                    black_killed_by_police NUMERIC,
                    pacific_islander_killed_by_police NUMERIC,
                    american_indian_or_alaska_native_killed_by_police NUMERIC,
                    hispanic_killed_by_police NUMERIC,
                    white_killed_by_police NUMERIC,
                    unknown_race_killed_by_police NUMERIC,
                    total_killed_by_police NUMERIC,
                    avg_annual_police_homicide_rate NUMERIC,
                    avg_annual_police_homicide_rate_black NUMERIC,
                    avg_annual_police_homicide_rate_white NUMERIC,
                    avg_annual_police_homicide_rate_hispanic NUMERIC,
                    black_white_dissimilarity_index_2010 NUMERIC,
                    black_white_disparity NUMERIC,
                    hispanic_white_disparity NUMERIC,
                    murder_and_nonnegligent_manslaughter NUMERIC,
                    murder_rate NUMERIC,
                    avg_violent_crimes_reported_2013_2017 NUMERIC,
                    violent_crime_rate NUMERIC,
                    estimated_avg_arrests_per_year NUMERIC,
                    killings_by_police_per_10k_arrests NUMERIC,
                    CONSTRAINT fk_police_violence_city FOREIGN KEY (city_id) REFERENCES city(city_id),
                    CONSTRAINT fk_police_violence_state FOREIGN KEY (state_id) REFERENCES state(state_id)
                )
            """)
        )
        print("Table created successfully.")
        for index, row in deaths_arrests.iterrows():
            conn.execute(
                text("""
                    INSERT INTO police_violence (
                        dept_name,
                        city_id,
                        state_id,
                        black_killed_by_police,
                        hispanic_killed_by_police,
                        american_indian_or_alaska_native_killed_by_police,
                        asian_killed_by_police,
                        pacific_islander_killed_by_police,
                        white_killed_by_police,
                        unknown_race_killed_by_police,
                        total_killed_by_police,
                        black_white_dissimilarity_index_2010,
                        murder_and_nonnegligent_manslaughter,
                        murder_rate,
                        avg_annual_police_homicide_rate,
                        avg_annual_police_homicide_rate_black,
                        avg_annual_police_homicide_rate_white,
                        avg_annual_police_homicide_rate_hispanic,
                        black_white_disparity,
                        hispanic_white_disparity,
                        avg_violent_crimes_reported_2013_2017,
                        violent_crime_rate,
                        estimated_avg_arrests_per_year,
                        killings_by_police_per_10k_arrests
                    ) VALUES (
                        :dept_name,
                        :city_id,
                        :state_id,
                        :black_killed_by_police,
                        :hispanic_killed_by_police,
                        :american_indian_or_alaska_native_killed_by_police,
                        :asian_killed_by_police,
                        :pacific_islander_killed_by_police,
                        :white_killed_by_police,
                        :unknown_race_killed_by_police,
                        :total_killed_by_police,
                        :black_white_dissimilarity_index_2010,
                        :murder_and_nonnegligent_manslaughter,
                        :murder_rate,
                        :avg_annual_police_homicide_rate,
                        :avg_annual_police_homicide_rate_black,
                        :avg_annual_police_homicide_rate_white,
                        :avg_annual_police_homicide_rate_hispanic,
                        :black_white_disparity,
                        :hispanic_white_disparity,
                        :avg_violent_crimes_reported_2013_2017,
                        :violent_crime_rate,
                        :estimated_avg_arrests_per_year,
                        :killings_by_police_per_10k_arrests
                    )
                """),
                {
                    "city_id": row['city_id'],
                    "state_id": row['state_id'],
                    "dept_name": row['dept_name'],
                    "black_killed_by_police": float(row['black_killed_by_police']),
                    "hispanic_killed_by_police": float(row['hispanic_killed_by_police']),
                    "american_indian_or_alaska_native_killed_by_police": float(row['american_indian_or_alaska_native_killed_by_police']),
                    "asian_killed_by_police": float(row['asian_killed_by_police']),
                    "pacific_islander_killed_by_police": float(row['pacific_islander_killed_by_police']),
                    "white_killed_by_police": float(row['white_killed_by_police']),
                    "unknown_race_killed_by_police": float(row['unknown_race_killed_by_police']),
                    "total_killed_by_police": float(row['total_killed_by_police']),
                    "black_white_dissimilarity_index_2010": float(row['black_white_dissimilarity_index_2010']),
                    "murder_and_nonnegligent_manslaughter": float(row['murder_and_nonnegligent_manslaughter']),
                    "murder_rate": float(row['murder_rate']),
                    "avg_annual_police_homicide_rate": float(row['avg_annual_police_homicide_rate']),
                    "avg_annual_police_homicide_rate_black": float(row['avg_annual_police_homicide_rate_black']),
                    "avg_annual_police_homicide_rate_white": float(row['avg_annual_police_homicide_rate_white']),
                    "avg_annual_police_homicide_rate_hispanic": float(row['avg_annual_police_homicide_rate_hispanic']),
                    "black_white_disparity": float(row['black_white_disparity']),
                    "hispanic_white_disparity": float(row['hispanic_white_disparity']),
                    "avg_violent_crimes_reported_2013_2017": float(row['avg_violent_crimes_reported_2013_2017']),
                    "violent_crime_rate": float(row['violent_crime_rate']),
                    "estimated_avg_arrests_per_year": float(row['estimated_avg_arrests_per_year']),
                    "killings_by_police_per_10k_arrests": float(row['killings_by_police_per_10k_arrests'])
                }
            )
        print('Data loaded successfully.')
    except Exception as e:
        print(f"Error: {e}")

Table created successfully.
Data loaded successfully.


### 'Violent Crimes and Arrests' table

In [116]:
deaths_arrests = pd.read_csv('data/deaths_arrests.csv')

deaths_arrests.drop(deaths_arrests.columns[[3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,36,37,38,44,45]],axis=1,inplace=True)

violent_crimes = pd.melt(deaths_arrests, id_vars=['City', 'State', 'PD'],
                    value_vars=[col for col in deaths_arrests.columns if 'Violent crimes' in col],
                    var_name='Crime_Type',
                   value_name='violent_crimes')

arrests = pd.melt(deaths_arrests,id_vars=['City','State','PD'],
                  value_vars=[col for col in deaths_arrests.columns if 'Arrest' in col],
                  var_name='Arrests_Type',
                  value_name='arrests')

violent_crimes['year'] = violent_crimes['Crime_Type'].str.extract(r'(\d{4})')
arrests['year'] = arrests['Arrests_Type'].str.extract(r'(\d{4})')

merged_df = pd.merge(violent_crimes, arrests, on=['City','State','PD','year'])
merged_df = merged_df.drop(columns=['Crime_Type','Arrests_Type'])
merged_df = merged_df.rename(columns = {'City':'city_name','State':'state_name','PD':'dept_name'})
merged_df['year'] = pd.to_datetime(merged_df['year'], format='%Y')
violent_crimes_arrests=pd.merge(merged_df,cities_df,on='city_name',how='left')
violent_crimes_arrests=violent_crimes_arrests.dropna(subset=['city_id','state_id'])
violent_crimes_arrests['city_id'] = violent_crimes_arrests['city_id'].astype(int)
violent_crimes_arrests['state_id'] = violent_crimes_arrests['state_id'].astype(int)
violent_crimes_arrests.tail()

with engine.connect() as conn:
    try:
        conn.execute(
            text("""   
                CREATE TABLE IF NOT EXISTS violent_crimes_arrests (
                    violent_crimes_arrests_id SERIAL PRIMARY KEY,
                    city_id INT NOT NULL,
                    state_id INT NOT NULL,
                    dept_name VARCHAR(50),
                    year DATE,
                    violent_crimes NUMERIC,
                    arrests NUMERIC,
                    CONSTRAINT fk_violent_crimes_arrests_city FOREIGN KEY (city_id) REFERENCES city (city_id),
                    CONSTRAINT fk_violent_crimes_arrests_state FOREIGN KEY (state_id) REFERENCES state (state_id)
                ) 
            """)
        )
        print("Table created successfully.")
        for index, row in violent_crimes_arrests.iterrows():
            conn.execute(
                text("""
                    INSERT INTO violent_crimes_arrests (
                        city_id,
                        state_id,
                        dept_name,
                        year,
                        violent_crimes,
                        arrests
                    ) VALUES (
                        :city_id,
                        :state_id,
                        :dept_name,
                        :year,
                        :violent_crimes,
                        :arrests
                    )
                """),
                {
                    "city_id": row['city_id'],
                    "state_id": row['state_id'],
                    "dept_name": row['dept_name'],
                    "year": row['year'],
                    "violent_crimes": float(row['violent_crimes']),
                    "arrests": float(row['arrests'])
                }
            )
        print('Data loaded successfully.')
    except Exception as e:
        print(f"Error: {e}")

Table created successfully.
Data loaded successfully.


## 'Poverty Demographics' table

In [141]:
poverty_df=pd.read_csv('data/demographics/poverty_census_bureau.csv')

new_column_names = {
    'Estimate!!Total!!Population for whom poverty status is determined': 'total_impoverished', 
    'Estimate!!Percent below poverty level!!Population for whom poverty status is determined': 'percent_impoverished',
}

poverty_df.rename(columns=new_column_names, inplace=True)
poverty_df = poverty_df[['Geographic Area Name','total_impoverished', 'percent_impoverished']]
poverty_df[['city_name', 'state_name']] = poverty_df['Geographic Area Name'].str.split(', ', expand=True, n=1)
poverty_df['state_abbreviation'] = poverty_df['Geographic Area Name'].str.extract(r',\s*([A-Z]{2})')
poverty_df = poverty_df.drop(columns=['state_name','Geographic Area Name'])
poverty_df=pd.merge(poverty_df,cities_df,on='city_name',how='left')
poverty_df=poverty_df.dropna(subset=['state_id','city_id'])
poverty_df['city_id'] = poverty_df['city_id'].astype(int)
poverty_df['state_id'] = poverty_df['state_id'].astype(int)
        
with engine.connect() as conn:
    try:
        conn.execute(
            text("""
                CREATE TABLE IF NOT EXISTS poverty_demographics (
                    poverty_demographics_id SERIAL PRIMARY KEY,
                    city_id INT NOT NULL,
                    state_id INT NOT NULL,
                    total_impoverished NUMERIC,
                    percent_impoverished NUMERIC
                )
            """)
        )
        print("'Poverty demographics' table created successfully.")
        for index, row in poverty_df.iterrows():
            conn.execute(
                text("""
                    INSERT INTO poverty_demographics (
                        city_id,
                        state_id,
                        total_impoverished,
                        percent_impoverished
                    ) VALUES (
                        :city_id,
                        :state_id,
                        :total_impoverished,
                        :percent_impoverished
                    )
                """),
                {
                    "city_id": row['city_id'],
                    "state_id": row['state_id'],
                    "total_impoverished": float(row['total_impoverished']),
                    "percent_impoverished": float(row['percent_impoverished'])
                }
            )
        print("'Data inserted into 'poverty demographics' successfully.")
    except Exception as e: 
        print(f"Error: {e}")

'Poverty demographics' table created successfully.
'Data inserted into 'poverty demographics' successfully.


### 'Political Demographics' table

In [156]:
political_demographics=pd.read_csv('data/demographics/politics_538.csv')
political_demographics=political_demographics.rename(columns={
    'Area':'area',
    'Republican Vote Share':'republican_vote_share',
    'Partisan Segregation':'partisan_segregation',
})
remove_strings = ['greater', 'area', 'city of']
for string in remove_strings:
    political_demographics['area'] = political_demographics['area'].str.replace(string, '', case=False)
df_split = political_demographics['area'].str.split(',', expand=True)
political_demographics['city_name'] = df_split[0].str.strip()  
political_demographics['state_name'] = df_split[1].str.strip()  
political_demographics = political_demographics.drop(columns=['area'])
political_demographics = pd.merge(political_demographics,cities_df,on='city_name',how='left')
political_demographics = political_demographics.dropna(subset=['city_id','state_id'])
political_demographics['city_id'] = political_demographics['city_id'].astype(int)
political_demographics['state_id'] = political_demographics['state_id'].astype(int)

with engine.connect() as conn:
    try:
        conn.execute(
            text("""
                CREATE TABLE IF NOT EXISTS political_demographics (
                    id SERIAL PRIMARY KEY,
                    city_id INT NOT NULL,
                    state_id INT NOT NULL,
                    republican_vote_share NUMERIC,
                    partisan_segregation NUMERIC
                )
            """)
        )
        print("'Political Demographics' table created successfully")
        for index, row in political_demographics.iterrows():
            conn.execute(
                text("""
                    INSERT INTO political_demographics (
                        city_id,
                        state_id,
                        republican_vote_share,
                        partisan_segregation
                    ) VALUES (
                        :city_id,
                        :state_id,
                        :republican_vote_share,
                        :partisan_segregation
                    )
                """),
                {
                    "city_id": row['city_id'],
                    "state_id": row['state_id'],
                    "republican_vote_share": float(row['republican_vote_share']),
                    "partisan_segregation": float(row['partisan_segregation'])
                }
            )
        print("Data loaded into 'political demographics' table successfully")
    except Exception as e:
        print(f"Error: {e}")

'Political Demographics' table created successfully
Data loaded into 'political demographics' table successfully


### 'Police Killings' table

In [185]:
police_killings = pd.read_csv('data/police_killings_MPV.csv')
shootings_wash_post = pd.read_csv('data/shootings_wash_post.csv')

police_killings = pd.DataFrame(police_killings[["Victim's name","Victim's age","Victim's gender",
                                               "Victim's race","City","State","Date of Incident (month/day/year)",
                                               "Agency responsible for death","Cause of death","A brief description of the circumstances surrounding the death",
                                              "Official disposition of death (justified or other)","Criminal Charges?","Symptoms of mental illness?",
                                              "Unarmed/Did Not Have an Actual Weapon","Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)",
                                              "Alleged Threat Level (Source: WaPo)","Fleeing (Source: WaPo)","Body Camera (Source: WaPo)",
                                              "Off-Duty Killing?"]])

police_killings = police_killings.rename(columns = {
    "Victim's name":'victim_name',
    "Victim's age":'victim_age',
    "Victim's gender":'victim_gender',
    "Victim's race":'victim_race',
    "City":'city_name',
    'State':'state_abbreviation',
    'Date of Incident (month/day/year)':'date_of_incident',
    'Agency responsible for death':'dept_responsible', 
    'Cause of death':'cause_of_death',
    'A brief description of the circumstances surrounding the death':'description',
    'Official disposition of death (justified or other)':'official_disposition',
    'Criminal Charges?':'criminal_charges',
    'Symptoms of mental illness?':'symptoms_mental_illness',
    'Unarmed/Did Not Have an Actual Weapon':'armed',
    'Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)':'alleged_weapon',
    'Alleged Threat Level (Source: WaPo)':'alleged_threat_level',
    'Fleeing (Source: WaPo)':'fleeing',
    'Body Camera (Source: WaPo)':'body_camera', 
    'Off-Duty Killing?':'off_duty'
})

shootings_wash_post=shootings_wash_post.drop(columns=['id'])
shootings_wash_post = shootings_wash_post.rename(columns = {
    'name':'victim_name',
    'city': 'city_name',
    'state': 'state_abbreviation',
    'age':'victim_age',
    'gender':'victim_gender',
    'race':'victim_race',
    'flee':'fleeing',
    'signs_of_mental_illness':'symptoms_mental_illness',
    'manner_of_death':'cause_of_death',
    'date':'date_of_incident',
    'threat_level':'alleged_threat_level'
})

concat_df = pd.concat([police_killings,shootings_wash_post],ignore_index=True)
police_killings=pd.merge(concat_df,cities_df,on='city_name',how='left')
police_killings=police_killings.dropna(subset=['city_id','state_id'])
police_killings['city_id']=police_killings['city_id'].astype(int)
police_killings['state_id']=police_killings['state_id'].astype(int)
police_killings['date_of_incident'] = pd.to_datetime(police_killings['date_of_incident'], format='mixed')
police_killings['victim_age'] = police_killings['victim_age'].replace('Unknown', np.nan)
police_killings['victim_age'] = police_killings['victim_age'].str.replace(r'[^0-9.]', '', regex=True)


with engine.connect() as conn:
    try:
        conn.execute(
            text("""
                CREATE TABLE IF NOT EXISTS police_killings (
                    police_killing_id SERIAL PRIMARY KEY,
                    city_id INT NOT NULL,
                    state_id INT NOT NULL,
                    victim_name VARCHAR(100),
                    victim_age NUMERIC,
                    victim_gender VARCHAR(50),
                    victim_race VARCHAR(50),
                    date_of_incident DATE,
                    dept_responsible VARCHAR(255),
                    cause_of_death VARCHAR(50),
                    official_disposition VARCHAR(255),
                    alleged_weapon VARCHAR(50),
                    alleged_threat_level VARCHAR(50),
                    fleeing VARCHAR(50),
                    body_camera VARCHAR(50)
                )
            """)
        )
        print("'Police Killings' table created successfully.")
        for index, row in police_killings.iterrows():
            conn.execute(
                text("""
                    INSERT INTO police_killings (
                        city_id,
                        state_id,
                        victim_name,
                        victim_age,
                        victim_gender,
                        victim_race,
                        date_of_incident,
                        dept_responsible,
                        cause_of_death,
                        official_disposition,
                        alleged_weapon,
                        alleged_threat_level,
                        fleeing,
                        body_camera
                    ) VALUES (
                        :city_id,
                        :state_id,
                        :victim_name,
                        :victim_age,
                        :victim_gender,
                        :victim_race,
                        :date_of_incident,
                        :dept_responsible,
                        :cause_of_death,
                        :official_disposition,
                        :alleged_weapon,
                        :alleged_threat_level,
                        :fleeing,
                        :body_camera
                    )
                """),
                {
                    "city_id": row['city_id'],
                    "state_id": row['state_id'],
                    "victim_name": row['victim_name'],
                    "victim_age": float(row['victim_age']),
                    "victim_gender": row['victim_gender'],
                    "victim_race": row['victim_race'],
                    "date_of_incident": row['date_of_incident'],
                    "dept_responsible": row['dept_responsible'],
                    "cause_of_death": row['cause_of_death'],
                    "official_disposition": row['official_disposition'],
                    "alleged_weapon": row['alleged_weapon'],
                    "alleged_threat_level": row['alleged_threat_level'],
                    "fleeing": row['fleeing'],
                    "body_camera": row['body_camera']
                }
            )
        print("Data loaded into 'police_killings' table successfully.")
    except Exception as e:
        print(f"Error: {e}")

/var/folders/1q/xp3s_g4562n85d_j0xm_sp080000gn/T/ipykernel_19842/1172696316.py:1: DtypeWarning: Columns (21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  police_killings = pd.read_csv('data/police_killings_MPV.csv')


'Police Killings' table created successfully.
Date loaded into 'police_killings' table successfully.


### 'Police Deaths' table

In [193]:
states_df.tail()

,state_id,state_name,state_abbreviation
46,47,Washington,WA
47,48,West Virginia,WV
48,49,Wisconsin,WI
49,50,Wyoming,WY
50,51,United States,US


In [194]:
police_deaths = pd.read_csv('data/police_deaths_538.csv')
police_deaths = police_deaths.drop(columns=['dept'])
police_deaths=police_deaths.drop(columns=['eow','cause'])
police_deaths=police_deaths.rename(columns={
    'person': 'officer_name',
    'state': 'state_abbreviation',
    'cause_short': 'cause_of_death',
    'date': 'date_of_incident',
    })
police_deaths=pd.merge(police_deaths,states_df,on='state_abbreviation',how='left')
police_deaths.tail()

# with engine.connect() as conn:
#     try:
#         conn.execute(
#             text("""
#                 CREATE TABLE police_deaths (
#                     id SERIAL PRIMARY KEY,
#                     officer_name VARCHAR(100),
#                     cause_of_death VARCHAR(25),
#                     year NUMERIC,
#                     canine VARCHAR(25),
#                     dept_name VARCHAR(50),
#                     state VARCHAR(25),
#                     CONSTRAINT fk_deaths_state FOREIGN KEY REFERENCES location (state)
#                 )
#             """)
#         )
#         print("Table created successfully")
#         for index, row in df_deaths.iterrows():
#             conn.execute(
#                 text("""
#                     INSERT INTO police deaths (
#                         officer_name,
#                         cause_of_death,
#                         year,
#                         canine,
#                         dept_name,
#                         state
#                     ) VALUES (
#                         :officer_name,
#                         :cause_of_death,
#                         :year,
#                         :canine,
#                         :dept_name,
#                         :state
#                     )
#                 """),
#                 {
#                     "officer_name": row['officer_name'],
#                     "cause_of_death": row['cause_of_death'],
#                     "year": float(row['year']),
#                     "canine": row['canine'],
#                     "dept_name": row['dept_name'],
#                     "state": row['state']
#                 }
#             )
#         print("Data loaded successfully")
#     except Exception as e:
#         print(f"Error: {e}")

,officer_name,cause_of_death,date_of_incident,year,canine,dept_name,state_abbreviation,state_id,state_name
22795,K9 Bruno,Accidental,2016-06-12,2016,True,Amarillo Police Department,TX,NaN,NaN
22796,K9 Lazer,Heat exhaustion,2016-06-20,2016,True,United States Department of Homeland Security ...,US,NaN,NaN
22797,K9 Tyson,Heat exhaustion,2016-06-27,2016,True,Fountain County Sheriff's Office,IN,NaN,NaN
22798,K9 Credo,Gunfire (Accidental),2016-06-28,2016,True,Long Beach Police Department,CA,NaN,NaN
22799,K9 Roscoe,Automobile accident,2016-07-01,2016,True,Emmett Police Department,ID,NaN,NaN


## 'Educational Demographics' table

In [ ]:
                # CREATE TABLE education_demographics (
                #     less_than_high_school
#                    9_12_grade
#                    some_college_or_associates
#                    bachelors
#                    graduate_or_advanced
#                     state_id VARCHAR(50),
#                     CONSTRAINT fk_demographics_state FOREIGN KEY (state_id) REFERENCES state (state_id)
#                 )
#             """)
#         print("'Demographics' table created successfully.")
#         for index, row in 